In [1]:
import collections
from nltk import ngrams
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string

In [2]:
 full_corpus = pd.read_csv('/Users/User/221project/data/party_data2.csv', header=None, names = ['label', 'tweet'], usecols = [1, 2], nrows=80000)

In [3]:
train = full_corpus[0:10000]
test = full_corpus[10000:20000]
stopwords = nltk.corpus.stopwords.words('english')

In [4]:
def getData(df):
    labels = []
    tweets = []

    for index, column in df.iterrows():
        label = column[0]
        tweet = column[1]
        tweets.append(tweet)
        if label == 'dem':
            labels.append(1)
        else:
            labels.append(0)

    return tweets, np.array(labels)

In [5]:
#lemmatizing
word_lemmatizer = WordNetLemmatizer()
def lemmatization(tokenized_words):
    lemmatized_text = [word_lemmatizer.lemmatize(word)for word in tokenized_words]
    return ' '.join(lemmatized_text)


def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    message = message.split()
    message = lemmatization(message)
    message = message.split()
    result = []
    for i in range(0, len(message)-1):
        result.append((message[i], message[i+1]))  
   
    return result
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    D = collections.defaultdict(int)
    Count = collections.defaultdict(int)
    index = 0;
    for message in messages:
        for bigram in get_words(message):
            if bigram[0] in stopwords and bigram[1] in stopwords:
                continue
            Count[bigram] += 1
            if Count[bigram] == 1:
                D[bigram] = index
                index += 1
    return D
    # *** END CODE HERE ***


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    F = np.zeros((len(messages), len(word_dictionary)))
    for row, message in enumerate(messages):
        words = get_words(message)
        for word in words:
            if word in word_dictionary:
                F[row, word_dictionary[word]] = words.count(word)

    return F
    # *** END CODE HERE ***


def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    phi_j_y0 = np.ones(matrix.shape[1]) * 1.0
    phi_j_y1 = np.ones(matrix.shape[1]) * 1.0


    for wordk in range(matrix.shape[1]):
        for i in range(matrix.shape[0]):
            if labels[i] == 1 and matrix[i, wordk] >= 1:
                phi_j_y1[wordk] += matrix[i, wordk]

            if labels[i] == 0 and matrix[i, wordk] >= 1:
                phi_j_y0[wordk] += matrix[i, wordk]

        phi_j_y1[wordk] /= (matrix.shape[1] + np.sum(matrix[labels==1, :]))

        phi_j_y0[wordk] /= (matrix.shape[1] + np.sum(matrix[labels==0, :]))


    phi_y = sum(labels) * 1.0 / len(labels)

    return (phi_j_y0, phi_j_y1, phi_y)
    # *** END CODE HERE ***


def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    phi_j_y0, phi_j_y1, phi_y = model

    p1 = np.zeros(matrix.shape[0])
    p0 = np.zeros(matrix.shape[0])
    for i in range(matrix.shape[0]):
        xGivY1 = 0
        xGivY0 = 0
        for d in range(matrix.shape[1]):
            xGivY1 += matrix[i][d] * (np.log(phi_j_y1[d]) + np.log(phi_y))
            xGivY0 += matrix[i][d] * (np.log(phi_j_y0[d]) + np.log(1-phi_y))

        p1[i] = xGivY1 - (xGivY1 + xGivY0)
        p0[i] = xGivY0 - (xGivY1 + xGivY0)

    predictions = np.zeros(matrix.shape[0])
    for i in range(predictions.shape[0]):
        if p1[i] > p0[i]:
            predictions[i] = 1
        else:
            predictions[i] = 0

    return predictions
    # *** END CODE HERE ***


def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: A list of the top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    phi_j_y0, phi_j_y1, phi_y = model
    indications = [ np.log(phi_j_y1[i]/phi_j_y0[i]) for i in range(phi_j_y1.shape[0])]
    indications = sorted(range(len(indications)), key=lambda i: indications[i], reverse=True)[:5]
    top5 = []
    for i in indications:
        for word, index in dictionary.items():
            if(index == i):
                top5.append(word)

    return top5
    # *** END CODE HERE ***



def main():
    train_messages, train_labels = getData(train)
    test_messages, test_labels = getData(test)

    dictionary = create_dictionary(train_messages)
    train_matrix = transform_text(train_messages, dictionary)

    
    test_matrix = transform_text(test_messages, dictionary)

    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)


    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

    print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

    top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

    print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

    
main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/User/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-c293e03db4b1>", line 224, in <module>
    main()
  File "<ipython-input-5-c293e03db4b1>", line 210, in main
    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)
  File "<ipython-input-5-c293e03db4b1>", line 124, in fit_naive_bayes_model
    phi_j_y1[wordk] /= (matrix.shape[1] + np.sum(matrix[labels==1, :]))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/User/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception,

KeyboardInterrupt: 